<a href="https://colab.research.google.com/github/satyaprasadnaidu/FMML-22B21A4448/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Which is the best combination of features in python?
The "best" combination of features in Python depends on what you're trying to accomplish. Python is a versatile programming language with a wide range of features and libraries that can be combined in various ways to solve different types of problems. Here are some common feature combinations in Python based on specific use cases:

Web Development:
Django or Flask for web frameworks. HTML/CSS for front-end development. Database integration with SQL or NoSQL databases. Libraries like SQLAlchemy, Django ORM, or PyMongo for database access. RESTful APIs using Flask-RESTful or Django REST framework. JavaScript for client-side interactivity (e.g., React, Vue.js).

Data Science and Machine Learning:
Libraries like NumPy, Pandas, and Matplotlib for data manipulation and visualization. Scikit-Learn for machine learning algorithms. Jupyter Notebooks for interactive data analysis. TensorFlow or PyTorch for deep learning. SciPy for scientific and technical computing. Tools like Seaborn and Plotly for advanced data visualization.

Automation and Scripting:
Standard library modules for file handling, regular expressions, and system operations. Third-party packages like Requests for making HTTP requests. Automation libraries like Selenium for web scraping or automation tasks. GUI development with libraries like Tkinter or PyQt.

DevOps and System Administration:
Shell scripting for automation. Configuration management tools like Ansible. Containerization with Docker. Web frameworks like Flask for building internal tools. Cloud services and SDKs for cloud automation (e.g., AWS Boto3, Google Cloud SDK).

Game Development:
Pygame for 2D game development. Pyglet or Panda3D for 3D game development. Libraries like NumPy for game physics. Game engines like Godot (with Python support) for more advanced projects.

Scientific Computing:
Libraries like SciPy, SymPy, and networkx for various scientific applications. Jupyter Notebooks for interactive research and documentation.

Data Analysis and Visualization:
Libraries like Plotly, Seaborn, and Bokeh for interactive data visualization. Statistical analysis with SciPy and statsmodels. Data cleaning and manipulation with Pandas.

Natural Language Processing (NLP):
NLTK and spaCy for NLP tasks. Libraries like Gensim for topic modeling. Transformers library (Hugging Face) for state-of-the-art NLP models.

IoT (Internet of Things):
MicroPython or CircuitPython for embedded systems. Libraries like Adafruit CircuitPython for hardware interfacing. MQTT or CoAP for IoT communication. Ultimately, the best combination of features in Python will depend on your specific project and requirements. Python's strength lies in its flexibility and rich ecosystem of libraries, which allows you to choose the right combination of tools and features to suit your needs.

2.How would you test or visualize four or more features?

The "best" combination of features in Python can vary greatly depending on the specific task or problem you're trying to solve. Python is a versatile programming language with a rich ecosystem of libraries and features, and the best combination will depend on your particular use case. Here are some popular combinations of features and libraries in Python for different purposes:

Web Development:
Flask or Django for web framework. SQLAlchemy or Django ORM for database access. HTML/CSS/JavaScript for frontend development. RESTful API design using Flask-RESTful or Django REST framework. Data Analysis and Scientific Computing:

NumPy for numerical computations

User 2. How would you test or visualize four or more features? ChatGPT Visualizing four or more features in your data can be challenging because it's difficult to represent multiple dimensions on a 2D or 3D plot. However, there are several techniques and tools you can use to explore and visualize high-dimensional data:

Pair Plots or Scatterplot Matrices:
Use libraries like Seaborn or Pandas to create pair plots or scatterplot matrices. These plots display pairwise relationships between features in a grid of subplots. While they can only show relationships between two features at a time, they are useful for identifying potential correlations or patterns.

Dimensionality Reduction:
Techniques like Principal Component Analysis (PCA) or t-Distributed Stochastic Neighbor Embedding (t-SNE) can help reduce the dimensionality of your data to two or three dimensions, which can then be easily visualized in a scatter plot.

Parallel Coordinates:
Parallel coordinates plots can be used to visualize high-dimensional data. Each feature is represented as a vertical axis, and a line is drawn connecting the values of each data point across these axes.

3D Scatter Plots:
When you have four features, you can create a 3D scatter plot. Matplotlib and Plotly are popular libraries for creating 3D plots. You can choose three features for the X, Y, and Z axes and represent the fourth feature with the color or size of the data points.

Heatmaps and Correlation Matrices:
Create heatmaps or correlation matrices to visualize the relationships between multiple features simultaneously. These visualizations can help you identify which features are strongly correlated with each other.

Interactive Visualizations:
Use interactive visualization libraries like Plotly or Bokeh. These libraries allow you to create interactive plots that can display multiple features and provide tooltips for detailed information.

3D Surface Plots:
If you have more than four features and want to visualize them together, you can consider 3D surface plots or contour plots. These can provide insights into the relationships between multiple features.

Hierarchical Clustering and Dendrogram:
Cluster your data using # clustering and create a dendrogram to visualize the hierarchical structure of your data. This can help identify groups or clusters of data points based on the features. Remember that the choice of visualization technique depends on the nature of your data and the specific insights you want to gain. Experiment with different visualization methods to find the one that best suits your needs for understanding the relationships between multiple features in your dataset.

3.Can you come up with your own features?
Yes, you can create your own features in Python by defining functions, classes, or modules that provide new functionality or extend existing ones. Here are a few ways to do this:

1. Custom Functions:
You can create custom functions to encapsulate specific operations or calculations. For example:

python def calculate_area_of_circle(radius): return 3.14159 * radius ** 2

2. Custom Classes:
You can define custom classes to represent objects with specific behaviors and attributes. For example:

python class Person: def init(self, name, age): self.name = name self.age = age

   def greet(self):
       return f"Hello, my name is {self.name} and I am {self.age} years old."


3. Custom Modules:
You can create custom modules by organizing related functions, classes, and variables into separate Python files. Then, you can import and use them in other scripts. For example:

python

mymodule.py
def custom_function(): return "This is a custom function."

class CustomClass: def init(self, value): self.value = value

In another script, you can import and use the custom module:

python import mymodule

result = mymodule.custom_function() obj = mymodule.CustomClass(42)

4. Decorators:
Decorators allow you to modify the behavior of functions or methods. You can create your own decorators to add custom functionality to existing functions. For example:

python def my_custom_decorator(func): def wrapper(args, kwargs): print("Before function execution") result = func(args, **kwargs) print("After function execution") return result return wrapper

@my_custom_decorator def my_function(): print("Inside my_function")

my_function()

5. Custom Context Managers:
You can define custom context managers using the contextlib module to manage resources or state within a with block. For example:

python from contextlib import contextmanager

@contextmanager def my_custom_context_manager(): print("Entering the context") yield print("Exiting the context")

with my_custom_context_manager(): print("Inside the context")

Few examples of how you can create custom features in Python to extend its functionality according to your specific needs. Python's flexibility and extensibility make it a powerful language for building custom solutions.

4.Will these features work for different classes other than 0 and 1?
It appears that you're asking about features or code related to a specific problem or context, but you haven't provided enough details for me to give you a specific answer. Features or code can work for different classes in Python depending on how they are implemented.

Here are some general principles to consider:
Modularity:
If you're writing code or defining features, try to make them as modular and reusable as possible. This will make it easier to adapt them for different classes or problems.

Generalization:
When designing features or code, think about whether they are specific to the classes 0 and 1 or if they can be generalized to handle a wider range of classes. For example, if you're working on a classification problem, make sure your code can handle multiple classes, not just 0 and 1.

Testing and Validation: Test your features or code with different class labels to ensure they work as expected. This will help you identify any issues or limitations when using them with different classes.

link textFlexibility:
Build flexibility into your code by using variables or parameters that can be easily adjusted to accommodate different classes or problem settings.

5.What will happen if we take more that two classes at a time?
In Python, when you refer to "classes," you're likely talking about object-oriented programming (OOP) classes, which are used to create objects and define their behavior. Python allows you to define and work with multiple classes in a single script or module, and there is no strict limit on the number of classes you can define in a Python program.

If you create and use more than two classes in your Python code, it's perfectly fine and a common practice in many software projects. Each class represents a blueprint for objects, and you can use these classes to model and manipulate different components or entities in your program.

Here are some key points to consider when working with multiple classes in Python:

Organization: Using multiple classes helps you organize your code, making it more modular and maintainable. Each class should have a specific responsibility or role within your program.

Inheritance: Python supports class inheritance, which means you can create subclasses that inherit attributes and methods from a parent class. This allows you to create hierarchies of classes to represent more complex relationships between objects.

Composition: You can create classes that use instances of other classes as attributes. This is known as composition and allows you to build complex objects by combining simpler ones.

Encapsulation: Classes provide a way to encapsulate data and behavior, which can help you achieve data hiding and abstraction.

Code Reusability: Using multiple classes allows you to reuse code across different parts of your program, which can save you time and reduce redundancy.

Here's a simple example demonstrating the use of three classes in Python:

python Copy code class Person: def init(self, name): self.name = name

def introduce(self):
    print(f"My name is {self.name}.")

class Car: def init(self, make, model): self.make = make self.model = model

def display_info(self):
    print(f"This is a {self.make} {self.model}.")

class Student: def init(self, name, student_id): self.name = name self.student_id = student_id

def display_info(self):
    print(f"Student: {self.name}, ID: {self.student_id}")

Creating instances of the classes
person = Person("Alice") car = Car("Toyota", "Camry") student = Student("Bob", "12345")

Using the methods of the classes
person.introduce() car.display_info() student.display_info() In this example, we have defined three classes: Person, Car, and Student, each with its own attributes and methods. You can create instances of these classes and use their methods to perform specific tasks or operations.

.